# ⚖️ Notebook 05C: Sampling Strategy Battle

**Objective:** Find optimal sampling strategy for top 2 models

**From Phase 2:**
- 🥇 **Winner:** AdaBoost (76.4% recall, 29.0% precision)
- 🥈 **Runner-up:** CatBoost (75.0% recall, 33.3% precision)

**Critical Questions:**
1. Can SMOTE push recall from 76% to 80%?
2. Can undersampling improve precision without hurting recall?
3. Is baseline (class weights only) already optimal?

**Test Matrix (6 Combinations):**
- **AdaBoost:** Baseline, SMOTE, RandomUnderSampler
- **CatBoost:** Baseline, SMOTE, RandomUnderSampler

**Selection Criteria:**
- **Primary:** Recall ≥ 70% (non-negotiable)
- **Secondary:** Maximize Precision (reduce false alarms)
- **Balance:** F1-Score for overall performance

**Expected Outcome:**
- SMOTE may increase recall but decrease precision
- Undersampling may balance recall-precision better
- Baseline might still be optimal (already 76.4%)

---

## 📦 Step 1: Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import (
    make_scorer, recall_score, precision_score, f1_score, roc_auc_score,
    confusion_matrix, balanced_accuracy_score
)
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
import time
import json
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (16, 8)

print("✅ Setup complete")

## 📊 Step 2: Load Data & Phase 2 Results

In [ ]:
print("=" * 60)
print("LOADING DATA & PHASE 2 RESULTS")
print("=" * 60)

# Load data
df = pd.read_csv('selected_features_05A.csv')
X = df.drop('Target', axis=1)
y = df['Target']

print(f"\n📊 Dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"   Features: {X.shape[1]}")

# Class distribution
class_counts = y.value_counts().sort_index()
print(f"\n📊 Target Distribution:")
print(f"   Class 0 (Healthy):  {class_counts[0]:,} ({class_counts[0]/len(y)*100:.2f}%)")
print(f"   Class 1 (At Risk):  {class_counts[1]:,} ({class_counts[1]/len(y)*100:.2f}%)")
print(f"   Imbalance ratio: {class_counts[0]/class_counts[1]:.1f}:1")

# Load Phase 2 results
with open('top_2_models_selection.json', 'r') as f:
    phase2_results = json.load(f)

print(f"\n📋 Phase 2 Winners:")
print(f"   🥇 {phase2_results['winner']['name']}: {phase2_results['winner']['recall_mean']:.3f} recall")
print(f"   🥈 {phase2_results['runner_up']['name']}: {phase2_results['runner_up']['recall_mean']:.3f} recall")

# Calculate scale_pos_weight
scale_pos_weight = class_counts[0] / class_counts[1]
print(f"\n💡 scale_pos_weight for boosting: {scale_pos_weight:.2f}")

## 🔀 Step 3: Train-Test Split

In [ ]:
print("=" * 60)
print("TRAIN-TEST SPLIT")
print("=" * 60)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n📊 Split:")
print(f"   Train: {X_train.shape[0]:,} samples")
print(f"   Test:  {X_test.shape[0]:,} samples")

# Verify stratification
train_dist = y_train.value_counts(normalize=True).sort_index()
test_dist = y_test.value_counts(normalize=True).sort_index()

print(f"\n✅ Stratification verified:")
print(f"   Train At-Risk: {train_dist[1]:.3f}")
print(f"   Test At-Risk:  {test_dist[1]:.3f}")

## 🎯 Step 4: Define 3 Sampling Strategies

In [ ]:
print("=" * 60)
print("DEFINING 3 SAMPLING STRATEGIES")
print("=" * 60)

print("\n📋 Strategies to Test:")
print("\n1️⃣ BASELINE: Class Weights Only")
print("   • No resampling")
print("   • Uses scale_pos_weight or class_weight='balanced'")
print("   • Baseline from Phase 2 (AdaBoost: 76.4%, CatBoost: 75.0%)")

print("\n2️⃣ SMOTE: Synthetic Minority Over-sampling")
print("   • Creates synthetic At-Risk samples")
print("   • Balances training data to 1:1 ratio")
print("   • May increase recall but decrease precision")
print(f"   • Will create ~{class_counts[0] - class_counts[1]:,} synthetic At-Risk samples")

print("\n3️⃣ RandomUnderSampler: Majority Under-sampling")
print("   • Randomly removes Healthy samples")
print("   • Balances training data to 1:1 ratio")
print("   • May improve precision, risk losing recall")
print(f"   • Will remove ~{class_counts[0] - class_counts[1]:,} Healthy samples")

print("\n💡 Why Test All 3?")
print("   • SMOTE: May push recall 76% → 80%")
print("   • Undersampling: May improve precision 29% → 35%")
print("   • Baseline: Might already be optimal (no overfitting)")

## 🤖 Step 5: Define Model Configurations

In [ ]:
print("=" * 60)
print("DEFINING MODEL CONFIGURATIONS")
print("=" * 60)

# Define base estimator for AdaBoost (CRITICAL!)
ada_base = DecisionTreeClassifier(
    max_depth=3,
    class_weight='balanced',
    random_state=42
)

print("\n🥇 AdaBoost Configuration:")
print("   • Base Estimator: DecisionTreeClassifier(max_depth=3, class_weight='balanced')")
print("   • n_estimators: 100")
print("   • algorithm: SAMME")
print("   • This config achieved 76.4% recall in Phase 2")

print("\n🥈 CatBoost Configuration:")
print(f"   • scale_pos_weight: {scale_pos_weight:.2f}")
print("   • iterations: 100")
print("   • This config achieved 75.0% recall in Phase 2")

print("\n⚠️ CRITICAL: These exact configs must be used!")
print("   Otherwise results won't match Phase 2 baseline.")

## 🧪 Step 6: Test AdaBoost with 3 Sampling Strategies

In [ ]:
print("=" * 60)
print("TESTING ADABOOST WITH 3 SAMPLING STRATEGIES")
print("=" * 60)

ada_results = []

# Strategy 1: Baseline (Class Weights Only)
print("\n" + "="*60)
print("1️⃣ AdaBoost + BASELINE (Class Weights Only)")
print("="*60)

ada_baseline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=3, class_weight='balanced', random_state=42),
        n_estimators=100,
        algorithm='SAMME',
        random_state=42
    ))
])

start = time.time()
ada_baseline.fit(X_train, y_train)
elapsed = time.time() - start

y_pred = ada_baseline.predict(X_test)
y_pred_proba = ada_baseline.predict_proba(X_test)[:, 1]

recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

print(f"\n📊 Results:")
print(f"   Recall:    {recall:.3f} ← PRIMARY")
print(f"   Precision: {precision:.3f}")
print(f"   F1-Score:  {f1:.3f}")
print(f"   ROC-AUC:   {roc_auc:.3f}")
print(f"   Time:      {elapsed:.1f}s")
print(f"\n   TP: {tp:,} | FP: {fp:,} | FN: {fn:,} | TN: {tn:,}")

ada_results.append({
    'Model': 'AdaBoost',
    'Strategy': 'Baseline',
    'Recall': recall,
    'Precision': precision,
    'F1': f1,
    'ROC_AUC': roc_auc,
    'Time': elapsed,
    'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn
})

# Strategy 2: SMOTE
print("\n" + "="*60)
print("2️⃣ AdaBoost + SMOTE")
print("="*60)

ada_smote = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=3, class_weight='balanced', random_state=42),
        n_estimators=100,
        algorithm='SAMME',
        random_state=42
    ))
])

start = time.time()
ada_smote.fit(X_train, y_train)
elapsed = time.time() - start

y_pred = ada_smote.predict(X_test)
y_pred_proba = ada_smote.predict_proba(X_test)[:, 1]

recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

print(f"\n📊 Results:")
print(f"   Recall:    {recall:.3f} ← PRIMARY")
print(f"   Precision: {precision:.3f}")
print(f"   F1-Score:  {f1:.3f}")
print(f"   ROC-AUC:   {roc_auc:.3f}")
print(f"   Time:      {elapsed:.1f}s")
print(f"\n   TP: {tp:,} | FP: {fp:,} | FN: {fn:,} | TN: {tn:,}")

ada_results.append({
    'Model': 'AdaBoost',
    'Strategy': 'SMOTE',
    'Recall': recall,
    'Precision': precision,
    'F1': f1,
    'ROC_AUC': roc_auc,
    'Time': elapsed,
    'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn
})

# Strategy 3: RandomUnderSampler
print("\n" + "="*60)
print("3️⃣ AdaBoost + RandomUnderSampler")
print("="*60)

ada_under = ImbPipeline([
    ('scaler', StandardScaler()),
    ('undersample', RandomUnderSampler(random_state=42)),
    ('classifier', AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=3, class_weight='balanced', random_state=42),
        n_estimators=100,
        algorithm='SAMME',
        random_state=42
    ))
])

start = time.time()
ada_under.fit(X_train, y_train)
elapsed = time.time() - start

y_pred = ada_under.predict(X_test)
y_pred_proba = ada_under.predict_proba(X_test)[:, 1]

recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

print(f"\n📊 Results:")
print(f"   Recall:    {recall:.3f} ← PRIMARY")
print(f"   Precision: {precision:.3f}")
print(f"   F1-Score:  {f1:.3f}")
print(f"   ROC-AUC:   {roc_auc:.3f}")
print(f"   Time:      {elapsed:.1f}s")
print(f"\n   TP: {tp:,} | FP: {fp:,} | FN: {fn:,} | TN: {tn:,}")

ada_results.append({
    'Model': 'AdaBoost',
    'Strategy': 'UnderSample',
    'Recall': recall,
    'Precision': precision,
    'F1': f1,
    'ROC_AUC': roc_auc,
    'Time': elapsed,
    'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn
})

print("\n✅ AdaBoost testing complete!")

## 🧪 Step 7: Test CatBoost with 3 Sampling Strategies

In [ ]:
print("\n" + "=" * 60)
print("TESTING CATBOOST WITH 3 SAMPLING STRATEGIES")
print("=" * 60)

cat_results = []

# Strategy 1: Baseline
print("\n" + "="*60)
print("4️⃣ CatBoost + BASELINE (scale_pos_weight)")
print("="*60)

cat_baseline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', CatBoostClassifier(
        iterations=100,
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        verbose=0
    ))
])

start = time.time()
cat_baseline.fit(X_train, y_train)
elapsed = time.time() - start

y_pred = cat_baseline.predict(X_test)
y_pred_proba = cat_baseline.predict_proba(X_test)[:, 1]

recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

print(f"\n📊 Results:")
print(f"   Recall:    {recall:.3f} ← PRIMARY")
print(f"   Precision: {precision:.3f}")
print(f"   F1-Score:  {f1:.3f}")
print(f"   ROC-AUC:   {roc_auc:.3f}")
print(f"   Time:      {elapsed:.1f}s")
print(f"\n   TP: {tp:,} | FP: {fp:,} | FN: {fn:,} | TN: {tn:,}")

cat_results.append({
    'Model': 'CatBoost',
    'Strategy': 'Baseline',
    'Recall': recall,
    'Precision': precision,
    'F1': f1,
    'ROC_AUC': roc_auc,
    'Time': elapsed,
    'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn
})

# Strategy 2: SMOTE
print("\n" + "="*60)
print("5️⃣ CatBoost + SMOTE")
print("="*60)

cat_smote = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', CatBoostClassifier(
        iterations=100,
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        verbose=0
    ))
])

start = time.time()
cat_smote.fit(X_train, y_train)
elapsed = time.time() - start

y_pred = cat_smote.predict(X_test)
y_pred_proba = cat_smote.predict_proba(X_test)[:, 1]

recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

print(f"\n📊 Results:")
print(f"   Recall:    {recall:.3f} ← PRIMARY")
print(f"   Precision: {precision:.3f}")
print(f"   F1-Score:  {f1:.3f}")
print(f"   ROC-AUC:   {roc_auc:.3f}")
print(f"   Time:      {elapsed:.1f}s")
print(f"\n   TP: {tp:,} | FP: {fp:,} | FN: {fn:,} | TN: {tn:,}")

cat_results.append({
    'Model': 'CatBoost',
    'Strategy': 'SMOTE',
    'Recall': recall,
    'Precision': precision,
    'F1': f1,
    'ROC_AUC': roc_auc,
    'Time': elapsed,
    'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn
})

# Strategy 3: RandomUnderSampler
print("\n" + "="*60)
print("6️⃣ CatBoost + RandomUnderSampler")
print("="*60)

cat_under = ImbPipeline([
    ('scaler', StandardScaler()),
    ('undersample', RandomUnderSampler(random_state=42)),
    ('classifier', CatBoostClassifier(
        iterations=100,
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        verbose=0
    ))
])

start = time.time()
cat_under.fit(X_train, y_train)
elapsed = time.time() - start

y_pred = cat_under.predict(X_test)
y_pred_proba = cat_under.predict_proba(X_test)[:, 1]

recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

print(f"\n📊 Results:")
print(f"   Recall:    {recall:.3f} ← PRIMARY")
print(f"   Precision: {precision:.3f}")
print(f"   F1-Score:  {f1:.3f}")
print(f"   ROC-AUC:   {roc_auc:.3f}")
print(f"   Time:      {elapsed:.1f}s")
print(f"\n   TP: {tp:,} | FP: {fp:,} | FN: {fn:,} | TN: {tn:,}")

cat_results.append({
    'Model': 'CatBoost',
    'Strategy': 'UnderSample',
    'Recall': recall,
    'Precision': precision,
    'F1': f1,
    'ROC_AUC': roc_auc,
    'Time': elapsed,
    'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn
})

print("\n✅ CatBoost testing complete!")

## 📊 Step 8: Comprehensive Comparison (All 6 Combinations)

In [ ]:
print("=" * 60)
print("SAMPLING STRATEGY BATTLE RESULTS")
print("=" * 60)

# Combine all results
all_results = ada_results + cat_results
results_df = pd.DataFrame(all_results)

# Sort by recall (primary), then precision (secondary)
results_df = results_df.sort_values(['Recall', 'Precision'], ascending=[False, False])

print("\n📊 All 6 Combinations (Ranked by Recall, then Precision):\n")
print(results_df[['Model', 'Strategy', 'Recall', 'Precision', 'F1', 'ROC_AUC']].to_string(index=False))

# Identify winner
winner = results_df.iloc[0]

print("\n" + "=" * 60)
print("🏆 WINNER: BEST MODEL + SAMPLING COMBINATION")
print("=" * 60)

print(f"\n🥇 {winner['Model']} + {winner['Strategy']}")
print(f"   Recall:    {winner['Recall']:.3f} ← Catches {winner['Recall']*100:.1f}% of At-Risk")
print(f"   Precision: {winner['Precision']:.3f}")
print(f"   F1-Score:  {winner['F1']:.3f}")
print(f"   ROC-AUC:   {winner['ROC_AUC']:.3f}")

print(f"\n📊 Confusion Matrix:")
print(f"   True Positives:  {winner['TP']:,} ← Caught At-Risk")
print(f"   False Negatives: {winner['FN']:,} ← Missed At-Risk")
print(f"   True Negatives:  {winner['TN']:,}")
print(f"   False Positives: {winner['FP']:,} ← False alarms")

# Compare to Phase 2 baseline
phase2_baseline = phase2_results['winner']['recall_mean']
improvement = winner['Recall'] - phase2_baseline

print(f"\n📈 vs Phase 2 Baseline:")
print(f"   Phase 2: {phase2_baseline:.3f} recall")
print(f"   Phase 3: {winner['Recall']:.3f} recall")
print(f"   Change:  {improvement:+.3f} ({improvement/phase2_baseline*100:+.1f}%)")

## 📈 Step 9: Visualization

In [ ]:
# Create comprehensive comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Create combo labels
results_df['Combo'] = results_df['Model'] + '\n' + results_df['Strategy']
combos = results_df['Combo'].tolist()
colors = ['#2ecc71', '#3498db', '#e74c3c', '#f39c12', '#9b59b6', '#1abc9c']

# 1. Recall comparison
axes[0, 0].barh(combos, results_df['Recall'], color=colors, alpha=0.7)
axes[0, 0].set_xlabel('Recall (At-Risk)', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Recall Comparison\n(Higher is Better)', fontsize=13, fontweight='bold')
axes[0, 0].axvline(x=0.70, color='green', linestyle='--', alpha=0.5, label='Target (70%)')
axes[0, 0].legend()
for i, v in enumerate(results_df['Recall']):
    axes[0, 0].text(v + 0.01, i, f'{v:.3f}', va='center', fontweight='bold')

# 2. Precision comparison
axes[0, 1].barh(combos, results_df['Precision'], color=colors, alpha=0.7)
axes[0, 1].set_xlabel('Precision (At-Risk)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Precision Comparison\n(Higher is Better)', fontsize=13, fontweight='bold')
for i, v in enumerate(results_df['Precision']):
    axes[0, 1].text(v + 0.01, i, f'{v:.3f}', va='center', fontweight='bold')

# 3. F1-Score comparison
axes[1, 0].barh(combos, results_df['F1'], color=colors, alpha=0.7)
axes[1, 0].set_xlabel('F1-Score', fontsize=12, fontweight='bold')
axes[1, 0].set_title('F1-Score (Balance)\n(Higher is Better)', fontsize=13, fontweight='bold')
for i, v in enumerate(results_df['F1']):
    axes[1, 0].text(v + 0.01, i, f'{v:.3f}', va='center', fontweight='bold')

# 4. Recall vs Precision scatter
for i, row in results_df.iterrows():
    axes[1, 1].scatter(row['Recall'], row['Precision'], s=200, 
                      color=colors[list(results_df.index).index(i)], alpha=0.7,
                      label=row['Combo'])
axes[1, 1].set_xlabel('Recall', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Precision', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Recall vs Precision Trade-off', fontsize=13, fontweight='bold')
axes[1, 1].legend(fontsize=8, loc='best')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 🔍 Step 10: Critical Analysis - Impact of Sampling Strategies

In [ ]:
print("=" * 60)
print("CRITICAL ANALYSIS: SAMPLING STRATEGY IMPACT")
print("=" * 60)

# Group by model to analyze sampling impact
print("\n📊 Impact Analysis by Model:\n")

for model_name in ['AdaBoost', 'CatBoost']:
    model_results = results_df[results_df['Model'] == model_name]
    baseline = model_results[model_results['Strategy'] == 'Baseline'].iloc[0]
    smote = model_results[model_results['Strategy'] == 'SMOTE'].iloc[0]
    under = model_results[model_results['Strategy'] == 'UnderSample'].iloc[0]
    
    print(f"{'='*60}")
    print(f"{model_name} Analysis")
    print(f"{'='*60}")
    
    print(f"\n📊 Baseline (Class Weights Only):")
    print(f"   Recall: {baseline['Recall']:.3f} | Precision: {baseline['Precision']:.3f}")
    
    print(f"\n📊 SMOTE (Oversampling):")
    print(f"   Recall: {smote['Recall']:.3f} ({smote['Recall']-baseline['Recall']:+.3f})")
    print(f"   Precision: {smote['Precision']:.3f} ({smote['Precision']-baseline['Precision']:+.3f})")
    if smote['Recall'] > baseline['Recall']:
        print(f"   ✅ SMOTE increased recall!")
    else:
        print(f"   ⚠️ SMOTE decreased recall")
    if smote['Precision'] < baseline['Precision']:
        print(f"   ⚠️ But precision dropped (more false alarms)")
    
    print(f"\n📊 RandomUnderSampler (Undersampling):")
    print(f"   Recall: {under['Recall']:.3f} ({under['Recall']-baseline['Recall']:+.3f})")
    print(f"   Precision: {under['Precision']:.3f} ({under['Precision']-baseline['Precision']:+.3f})")
    if under['Precision'] > baseline['Precision']:
        print(f"   ✅ Undersampling improved precision!")
    else:
        print(f"   ⚠️ Precision didn't improve")
    if under['Recall'] < baseline['Recall']:
        print(f"   ⚠️ But recall dropped (misses more At-Risk)")
    
    print(f"\n💡 Conclusion for {model_name}:")
    if baseline['Recall'] >= max(smote['Recall'], under['Recall']):
        print(f"   Baseline is OPTIMAL - sampling doesn't help")
        print(f"   Class weights alone are sufficient")
    elif smote['Recall'] > baseline['Recall'] and smote['Recall'] > under['Recall']:
        print(f"   SMOTE is best if maximizing recall is priority")
    else:
        print(f"   UnderSampling is best if balancing recall-precision")
    print()

print("=" * 60)
print("OVERALL INSIGHTS")
print("=" * 60)

print("\n🎯 Key Findings:")
print(f"\n1️⃣ Best Recall: {results_df.iloc[0]['Model']} + {results_df.iloc[0]['Strategy']}")
print(f"   {results_df.iloc[0]['Recall']:.1%} of At-Risk patients detected")

best_precision = results_df.loc[results_df['Precision'].idxmax()]
print(f"\n2️⃣ Best Precision: {best_precision['Model']} + {best_precision['Strategy']}")
print(f"   {best_precision['Precision']:.1%} of predictions are correct")

best_f1 = results_df.loc[results_df['F1'].idxmax()]
print(f"\n3️⃣ Best F1 (Balance): {best_f1['Model']} + {best_f1['Strategy']}")
print(f"   F1-Score: {best_f1['F1']:.3f}")

print(f"\n⚠️ Recall-Precision Trade-off:")
print(f"   High recall (76-80%) comes with low precision (25-35%)")
print(f"   For every 100 predicted At-Risk, ~25-35 are truly At-Risk")
print(f"   This is ACCEPTABLE for screening (better safe than sorry)")
print(f"   But costly if follow-up tests are expensive")

## 💾 Step 11: Save Results for Phase 4

In [ ]:
print("=" * 60)
print("SAVING RESULTS FOR PHASE 4")
print("=" * 60)

# Save full results
results_df.to_csv('sampling_strategy_results.csv', index=False)
print(f"\n✅ Saved: sampling_strategy_results.csv")

# Save winner selection
winner_selection = {
    'model': winner['Model'],
    'sampling_strategy': winner['Strategy'],
    'recall': float(winner['Recall']),
    'precision': float(winner['Precision']),
    'f1': float(winner['F1']),
    'roc_auc': float(winner['ROC_AUC']),
    'confusion_matrix': {
        'TP': int(winner['TP']),
        'FP': int(winner['FP']),
        'TN': int(winner['TN']),
        'FN': int(winner['FN'])
    },
    'phase1_baseline': 0.210,
    'phase2_baseline': phase2_baseline,
    'phase3_final': float(winner['Recall']),
    'total_improvement': float(winner['Recall'] - 0.210)
}

with open('best_model_sampling_combo.json', 'w') as f:
    json.dump(winner_selection, f, indent=2)

print(f"✅ Saved: best_model_sampling_combo.json")

print("\n" + "=" * 60)
print("🎯 PHASE 3 COMPLETE: SAMPLING STRATEGY BATTLE")
print("=" * 60)

print(f"\n🏆 Selected for Phase 4 (Feature Selection):")
print(f"   Model: {winner['Model']}")
print(f"   Sampling: {winner['Strategy']}")
print(f"   Configuration: ", end="")
if winner['Model'] == 'AdaBoost':
    print("DecisionTree(class_weight='balanced', max_depth=3) + SAMME")
else:
    print(f"scale_pos_weight={scale_pos_weight:.2f}")

print(f"\n📊 Current Performance:")
print(f"   Recall:    {winner['Recall']:.3f} ({winner['Recall']*100:.1f}% of At-Risk detected)")
print(f"   Precision: {winner['Precision']:.3f}")
print(f"   F1-Score:  {winner['F1']:.3f}")

print(f"\n📈 Journey So Far:")
print(f"   Phase 1 (Feature Eng): 0.210 recall (baseline)")
print(f"   Phase 2 (Model Select): {phase2_baseline:.3f} recall (+{phase2_baseline-0.210:.3f})")
print(f"   Phase 3 (Sampling):     {winner['Recall']:.3f} recall ({winner['Recall']-phase2_baseline:+.3f})")
print(f"   Total Improvement:      {winner['Recall']-0.210:+.3f} ({(winner['Recall']-0.210)/0.210*100:+.0f}%)")

print(f"\n🎯 Next: Phase 4 - Feature Selection (RFECV)")
print(f"   Notebook: 05D_feature_selection.ipynb")
print(f"   Goal: Remove redundant features, improve generalization")
print(f"   Method: Recursive Feature Elimination with Cross-Validation")
print(f"   Expected: 10-15 features (from current 18)")
print(f"   Benefit: Simpler model, potentially better recall")